# Accessing Conflict Data
- [Armed Conflict Location & Event Data Website](https://acleddata.com/#/dashboard) 
- [API Guide](https://developer.acleddata.com/rehd/cms/views/acled_api/documents/API-User-Guide.pdf)
- [Dictionary](https://acleddata.com/acleddatanew//wp-content/uploads/2021/11/ACLED_Data-Columns_v1_April-2019.pdf)
- [Codebook](https://acleddata.com/acleddatanew//wp-content/uploads/2021/11/ACLED_Codebook_v1_January-2021.pdf)
- [Full Documentation](https://acleddata.com/resources/#1643628716684-37d18c29-1234)


In [9]:
#importing relevant packages
%run '/Users/thomasadler/Desktop/futuristic-platipus/notebooks/ta_0.1_packages.py'

In [10]:
# import useful functions
%run '/Users/thomasadler/Desktop/futuristic-platipus/notebooks/ta_0.2_functions.py'

ModuleNotFoundError: No module named 'ta_0_ta_packages'

In [ ]:
#fetching hidden api key and username from desktop stored under 'conflict_api_key' and 'conflict_api_email'
%run /Users/thomasadler/Desktop/futuristic-platipus/keys.py

In [ ]:
#defining working directory
filepath = '/Users/thomasadler/Desktop/capstone_docs/'

In [ ]:
#setting API endpoint,
conflict_api_endpoint = "https://api.acleddata.com/acled/read"

In [ ]:
#Uganda ISO code is 800
uganda_iso = 800

In [ ]:
# sending post request
conflict_r = requests.get(f'{conflict_api_endpoint}?key={conflict_api_key}&email={conflict_api_email}&limit=0&iso={uganda_iso}.csv')

#saving as json data
data = conflict_r.json()

#extract events information
events = data['data']

#save to a dataframe
uganda_conflict = pd.DataFrame(events)

In [ ]:
#leaving raw dataset untouched
uganda_conflict_df= uganda_conflict.copy()

In [ ]:
#checking dataset
uganda_conflict_df.tail()

In [ ]:
#check data types
uganda_conflict_df.info()

In [ ]:
#convert to number
num_columns=['latitude', 'longitude', 'fatalities',]

for col in num_columns:
    uganda_conflict_df[[col]]=uganda_conflict_df[[col]].astype('float32')
    #check
    print(uganda_conflict_df[[col]].head())

#check
uganda_conflict_df.info()

In [ ]:
#convert to datetime
uganda_conflict_df['event_date']=pd.to_datetime(uganda_conflict_df['event_date'])

#check
uganda_conflict_df['event_date'].tail()

In [ ]:
#drop irrelevant columns
uganda_conflict_df=pd.DataFrame(uganda_conflict_df.drop(columns=['time_precision', 'event_id_cnty','event_id_no_cnty',\
                                                              'geo_precision','timestamp','year',\
                                                             'iso','iso3', 'region','country']))

#check current columns
uganda_conflict_df.info()

In [ ]:
#checking administrative levels match the water dataset
print('admin1 in the conflict dataset should be clean_adm1 in the water dataset, check with:',uganda_conflict_df['admin1'].head(1)[0])
print('admin2 in the conflict dataset should be clean_adm2 in the water dataset, check with:',uganda_conflict_df['admin2'].head(1)[0])
print('admin3 in the conflict dataset should be clean_adm3 in the water dataset, check with:',uganda_conflict_df['admin3'].head(1)[0])
print('location in the conflict dataset should be clean_adm4 in the water dataset, check with:',uganda_conflict_df['location'].head(1)[0])

In [ ]:
#replacing column name to match other datasets
uganda_conflict_df.rename(columns={'admin1': 'clean_adm1','admin2': 'clean_adm2','admin3': 'clean_adm3', 'location':'clean_adm4'}, inplace=True)

In [ ]:
#checking null values and duplicated rows and columns
print(uganda_conflict_df.isna().sum().sum()>0,\
      uganda_conflict_df.duplicated().sum()>0,\
      uganda_conflict_df.T.duplicated().sum()>0)

In [ ]:
#export to cleaned dataset to csv
uganda_conflict_df.to_csv(filepath + 'uganda_conflict_df_clean.csv')